In [ ]:
import sdf_helper as sh
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

# # 设定数据文件的路径和文件名模式  
base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch-set2/epoch3d/ju2024a'
file_prefix = 'weight'  
file_suffix = '.sdf'

file_index =  36

file_name = f"{file_prefix}{file_index:04d}{file_suffix}"  
file_path = os.path.join(base_path, file_name)

# # 读取数据
# den_crit =  0.1
weight_Data = sh.getdata(file_path)
sh.list_variables(weight_Data)

weight_ph = weight_Data.Particles_Weight_subset_testp_Photon.data


In [ ]:
import sdf_helper as sh
import numpy as np
import os

base_path = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch-set2/epoch3d/ju2024a'
file_index = 36
file_path = os.path.join(base_path, f'idall{file_index:04d}.sdf')

data = sh.getdata(file_path)
sh.list_variables(data)

def get_data(name):
    arr = getattr(data, name).data
    return arr / 1e-6 if 'Grid_Particles' in name else arr

px_ph, py_ph, pz_ph = (get_data(f'Particles_P{d}_subset_testp_Photon') for d in ['x','y','z'])
x_ph, y_ph, z_ph = get_data('Grid_Particles_subset_testp_Photon')

px_ele, py_ele, pz_ele = (get_data(f'Particles_P{d}_subset_teste_Electron') for d in ['x','y','z'])
x_ele, y_ele, z_ele = get_data('Grid_Particles_subset_teste_Electron')

weight_ph, weight_ele = 1500, 1600

x_range = (10, 50)
y_range = None
z_range = None

def filter_range(x, y, z, xr, yr, zr):
    mask = np.ones_like(x, dtype=bool)
    if xr: mask &= (x >= xr[0]) & (x <= xr[1])
    if yr: mask &= (y >= yr[0]) & (y <= yr[1])
    if zr: mask &= (z >= zr[0]) & (z <= zr[1])
    return mask

mask_ph = filter_range(x_ph, y_ph, z_ph, x_range, y_range, z_range)
mask_ele = filter_range(x_ele, y_ele, z_ele, x_range, y_range, z_range)

Lx_ph = (y_ph[mask_ph] * pz_ph[mask_ph] - z_ph[mask_ph] * py_ph[mask_ph]) * weight_ph
Lx_ele = (y_ele[mask_ele] * pz_ele[mask_ele] - z_ele[mask_ele] * py_ele[mask_ele]) * weight_ele

# 打印筛选后的粒子坐标范围和对应角动量统计信息
def print_stats(particle_type, x, y, z, Lx, mask):
    print(f"{particle_type} 粒子筛选后范围及角动量统计：")
    print(f"x 范围: [{x[mask].min():.3f}, {x[mask].max():.3f}] μm")
    print(f"y 范围: [{y[mask].min():.3f}, {y[mask].max():.3f}] μm")
    print(f"z 范围: [{z[mask].min():.3f}, {z[mask].max():.3f}] μm")
    print(f"Lx 统计:")
    print(f"  最小值: {Lx.min():.3e}")
    print(f"  最大值: {Lx.max():.3e}")
    print(f"  平均值: {Lx.mean():.3e}")
    print(f"  标准差: {Lx.std():.3e}")
    print("-" * 40)

print_stats('光子', x_ph, y_ph, z_ph, Lx_ph, mask_ph)
print_stats('电子', x_ele, y_ele, z_ele, Lx_ele, mask_ele)
